# New Scaling API Demonstration

This notebook demonstrates the improved scaling API:
- `uw.apply_scaling()` context manager
- `uw.unwrap(expr, apply_scaling=True)` function with scaling parameter

In [1]:
import sys
sys.path.insert(0, '/Users/lmoresi/+Underworld/underworld-pixi-2/underworld3/src')

import underworld3 as uw

In [2]:
# Setup variables with units
mesh = uw.meshing.StructuredQuadBox(elementRes=(2, 2), minCoords=(0, 0), maxCoords=(1, 1))

temperature = uw.discretisation.MeshVariable("temp", mesh, 1, units="K", varsymbol=r"T")
temperature.set_reference_scaling(1500.0)

velocity = uw.discretisation.MeshVariable("vel", mesh, 2, units="cm/year", varsymbol=r"\mathbf{v}")
velocity.set_reference_scaling(5.0)

uw.pprint(f"Temperature scale factor: {temperature.scale_factor}")
uw.pprint(f"Velocity scale factor: {velocity.scale_factor}")

Structured box element resolution 2 2
Temperature scale factor: 0.00100000000000000
Velocity scale factor: 0.100000000000000


## Method 1: Function Parameter Approach

In [3]:
# Create expression
expr = uw.function.expression("advection", temperature[0,0] + velocity[0], "advection equation")
uw.pprint(f"Expression: {expr}")

# Normal unwrap
normal_result = uw.unwrap(expr)
uw.pprint(f"Normal unwrap: {normal_result}")

# Unwrap with scaling parameter
scaled_result = uw.unwrap(expr, apply_scaling=True)
uw.pprint(f"Scaled unwrap: {scaled_result}")

uw.pprint(f"Different results: {normal_result != scaled_result}")

Expression: UWexpression instance 5, advection
Normal unwrap: T (N.x, N.y) + v _0 (N.x, N.y)
Scaled unwrap: 0.001*T (N.x, N.y) + 0.1*v _0 (N.x, N.y)
Different results: True


## Method 2: Context Manager Approach

In [4]:
# Using apply_scaling context manager
with uw.apply_scaling():
    context_result = uw.unwrap(expr)
    uw.pprint(f"Context scaling: {context_result}")

# Verify both methods produce identical results
uw.pprint(f"Both methods identical: {scaled_result == context_result}")

Context scaling: 0.001*T (N.x, N.y) + 0.1*v _0 (N.x, N.y)
Both methods identical: True


## Testing with Different Parameters

In [5]:
# Test with different unwrap parameters
uw.pprint("Testing with different parameters:")

# Test keep_constants parameter
result1 = uw.unwrap(expr, keep_constants=False, apply_scaling=True)
uw.pprint(f"keep_constants=False: {result1}")

# Test return_self parameter  
result2 = uw.unwrap(expr, return_self=False, apply_scaling=True)
uw.pprint(f"return_self=False: {result2}")

# Test all parameters together
result3 = uw.unwrap(expr, keep_constants=False, return_self=False, apply_scaling=True)
uw.pprint(f"All parameters: {result3}")

Testing with different parameters:
keep_constants=False: 0.001*T (N.x, N.y) + 0.1*v _0 (N.x, N.y)
return_self=False: 0.001*T (N.x, N.y) + 0.1*v _0 (N.x, N.y)
All parameters: 0.001*T (N.x, N.y) + 0.1*v _0 (N.x, N.y)


## Summary

The new API provides two clean ways to apply scaling:

### Function Parameter (Recommended)
```python
# Explicit and clear
result = uw.unwrap(expr, apply_scaling=True)
```

### Context Manager
```python
# Useful for multiple operations
with uw.apply_scaling():
    result = uw.unwrap(expr)
```

Both methods:
- Support all original unwrap parameters
- Produce identical scaling results
- Maintain backward compatibility